In [13]:
import psi4
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
from rdkit.Chem import Descriptors
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

# Import CSV file containing fuels

In [14]:
df= pd.read_csv('ExtraMolecules.csv')
df

,ID,Fuel,Class,Sub-Class,Group,BP2,Uncertainty,Source,SMILES
0,326,"1,1-dimethoxyethane",Acetals,Aliphatic acetals,Alkyl acetals,336.695,1.85029,NIST,COC(C)OC
1,327,"2,4,6-trimethyl-1,3,5-trioxane",Acetals,Aliphatic acetals,Alkyl acetals,397.316,1.84010,NIST,CC1OC(C)OC(C)O1
2,328,dimethyl peroxide,Organic peroxides,Other peroxides,Other peroxides,270.414,11.08210,NIST,COOC
3,329,heptane-3-peroxol,Organic peroxides,Hydroperoxides,Aliphatic hydroperoxides,459.704,1.50806,NIST,CCCCC(CC)OO
4,330,propane-1-peroxol,Organic peroxides,Hydroperoxides,Aliphatic hydroperoxides,377.335,1.19363,NIST,CCCOO


# Generate SMILES column and add it to existing dataframe

In [15]:
smiles_list = df['SMILES']

# Define function that will optimize a molecule's 3D structure to obtain the structure with the lowest energy - Round 1
# **Important for desriptor calculations

In [16]:
def optimize(m):
    """ 
        Input: RDKit molecule object
        
        Optimizes molecular structure by applying the Merck molecular force field 94(MMFF94) before performing tasks:
        
        1. Calculates total number of atoms
        
        2. Coverts geometry of mol object to a string of xyz coordinates
    
    """
    m = Chem.AddHs(m)#add hydrogens to structure
    #Convert from 2D to 3D
    AllChem.EmbedMolecule(m,randomSeed=0xf00d,useExpTorsionAnglePrefs=True,useBasicKnowledge=True)#intialize molecular conformation
    AllChem.MMFFOptimizeMolecule(m,'MMFF94')#apply force field
    atoms = m.GetAtoms()
    string = "\n"
    for i, atom in enumerate(atoms):
        pos = m.GetConformer().GetAtomPosition(atom.GetIdx())
        string += "{} {} {} {}\n".format(atom.GetSymbol(), pos.x, pos.y, pos.z)
    string += "units angstrom\n"
    string += "symmetry c1\n"
    #-------------------------------------------
    totAtoms = m.GetNumAtoms()
    #-----------------------------------------------
    
    
    return totAtoms, string, m

# Calculate descriptors: Number of atoms, molecular weight, IC0, PJI3, SIC0, GATS1v, Wiener Index, Zagreb Index

In [17]:
from mordred import Autocorrelation,BaryszMatrix,CPSA


GATS2m = Autocorrelation.GATS (2, 'm')
#VR3_Dzv = BaryszMatrix.BaryszMatrix('v', 'VR3')#vdw volume
#VR3_Dzp =  BaryszMatrix.BaryszMatrix('p', 'VR3')#polarizvility
FPSA3 = CPSA.FPSA(3)
WNSA3 = CPSA.WNSA (3)
PNSA3 = CPSA.PNSA(3)
ATS3p = Autocorrelation.ATS(3, 'p')
RPCG = CPSA.RPCG()
RNCG = CPSA.RNCG()

GATS2m_list = []
# VR3_v_list = []
# VR3_p_list = []
FPSA3_list = []
WNSA3_list = []
PNSA3_list = []
ATS3p_list=[]
RPCG_list=[]
RNCG_list = []
#-------------------------
from mordred import BalabanJ, InformationContent, TopologicalCharge, MoRSE
from mordred import GravitationalIndex, HydrogenBond, AtomCount, BondCount, CPSA

balabanJ = BalabanJ.BalabanJ()
CIC3 = InformationContent.ComplementaryIC(order=3)
GGI4 = TopologicalCharge.TopologicalCharge(type='raw',order=4)
JGI1 = TopologicalCharge.TopologicalCharge(type='mean',order=1)
Mor10v = MoRSE.MoRSE(prop='v',distance=10)
Mor16se = MoRSE.MoRSE(prop='se',distance=16)
GRAVH = GravitationalIndex.GravitationalIndex(False,False)
HBondA = HydrogenBond.HBondAcceptor()
HBondD = HydrogenBond.HBondDonor()
nAtomHetero = AtomCount.AtomCount('Hetero')
nBondsA = BondCount.BondCount('aromatic', False)
CPS = CPSA.DPSA()

balaban = []
CIC3_list=[]
GGI4_list = []
JGI1_list = []
Mor10v_list= []
Mor16se_list = []
GRAVH_list = []
HBA_list = []
HBD_list = []
Hetero_list = []
AroBonds_list = []
CPS_list = []


#-----------------------------------------
from mordred import InformationContent
from mordred import GeometricalIndex
from mordred import Autocorrelation
from mordred import WienerIndex
from mordred import ZagrebIndex

IC0_calc = InformationContent.InformationContent(order=0)
petitjean_calc = GeometricalIndex.PetitjeanIndex3D()
SIC0_calc = InformationContent.StructuralIC(order=0)
GATS1v_calc = Autocorrelation.GATS(order=1,prop='v')
wiener_index = WienerIndex.WienerIndex()
zagreb_index1 = ZagrebIndex.ZagrebIndex(version = 1) 


IC0_list = [] # Information content index (neighborhood symmetry of 0-order)
petitjean_3D = [] #3D petitjean shape index
SIC0_list = []
GATS1v_list = []
wiener_list= [] #stores wiener indices
Z1_list = [] #stores zagreb1 indices



MW_list = [] #stores molecular weight
mol_form = [] #stores molecular formulas
N_atoms = [] #stores number of atoms
xyzgeom_list = [] #stores xyz coordinates


for smiles in smiles_list:

    mol = Chem.MolFromSmiles(smiles)
    MW = Descriptors.MolWt(mol)
    MW_list.append(MW)
    form = CalcMolFormula(mol) 
    mol_form.append(form)
    
    totN_atoms, xyzcoords, mol2 = optimize(mol)
    xyzgeom_list.append(xyzcoords)
    N_atoms.append(totN_atoms)
    
    ic0 = IC0_calc(mol2)
    IC0_list.append(ic0)
    petit_index = petitjean_calc(mol2)
    petitjean_3D.append(petit_index)
    sic0_index = SIC0_calc(mol2)
    SIC0_list.append(sic0_index)
    gats = GATS1v_calc(mol2)
    GATS1v_list.append(gats)
    wiener = wiener_index(mol2)
    wiener_list.append(wiener)
    Z1 = zagreb_index1(mol2)
    Z1_list.append(Z1)
    
    
    GRAVH_list.append(GRAVH(mol2))
    HBA_list.append(HBondA(mol2))
    HBD_list.append(HBondD(mol2))
    Hetero_list.append(nAtomHetero(mol2))
    AroBonds_list.append(nBondsA(mol2))
    CPS_list.append(CPS(mol2))
    
    
    balaban.append(balabanJ(mol2))
    CIC3_list.append(CIC3(mol2))
    GGI4_list.append(GGI4(mol2))
    JGI1_list.append(JGI1(mol2))
    Mor10v_list.append(Mor10v(mol2))
    Mor16se_list.append(Mor16se(mol2))
    
    GATS2m_list.append(GATS2m(mol2))
#     VR3_v_list.append(VR3_Dzv(mol2))
#     VR3_p_list.append(VR3_Dzp(mol2))
    FPSA3_list.append(FPSA3(mol2))
    WNSA3_list.append(WNSA3(mol2))
    PNSA3_list.append(PNSA3(mol2))
    ATS3p_list.append(ATS3p(mol2))
    RPCG_list.append(RPCG(mol2))
    RNCG_list.append(RNCG(mol2))

# Add columns with new properties to existing dataframe

In [18]:
df['Molecular Formula'] = mol_form
df['N_Atoms'] = N_atoms
df['Molecular Weight (g/mol)'] = MW_list
df['IC0'] = IC0_list
df['PJ3'] = petitjean_3D
df['SIC0'] = SIC0_list
df['GATS1v'] = GATS1v_list
df['Wiener'] = wiener_list
df['Z1'] = Z1_list


df['BalabanJ'] = balaban
df['CIC3'] = CIC3_list
df['GGI4'] = GGI4_list
df['JGI1'] = JGI1_list
df['Mor10v'] = Mor10v_list
df['Mor16se'] = Mor16se_list
df['GRAVH'] = GRAVH_list
df['HBondA'] =HBA_list
df['HBondD'] = HBD_list
df['HeteroA'] = Hetero_list
df['AromaticR'] = AroBonds_list
df['CPS'] = CPS_list

df['GATS2m'] = GATS2m_list
# df['VR3_Dzv'] = VR3_v_list
# df['VR3_Dzp'] = VR3_p_list
df['FPSA3']=FPSA3_list
df['WNSA3'] = WNSA3_list
df['PNSA3'] = PNSA3_list 
df['ATS3p'] = ATS3p_list 
df['RPCG'] = RPCG_list 
df['RNCG'] = RNCG_list 

df

,ID,Fuel,Class,Sub-Class,Group,BP2,Uncertainty,Source,SMILES,Molecular Formula,...,HeteroA,AromaticR,CPS,GATS2m,FPSA3,WNSA3,PNSA3,ATS3p,RPCG,RNCG
0,326,"1,1-dimethoxyethane",Acetals,Aliphatic acetals,Alkyl acetals,336.695,1.85029,NIST,COC(C)OC,C4H10O2,...,2,0,173.818431,1.128388,0.038517,-2.858090,-10.848476,21.707281,0.211493,0.490707
1,327,"2,4,6-trimethyl-1,3,5-trioxane",Acetals,Aliphatic acetals,Alkyl acetals,397.316,1.84010,NIST,CC1OC(C)OC(C)O1,C6H12O3,...,3,0,151.296393,0.912466,0.030933,-3.920698,-12.372305,41.060026,0.158624,0.320290
2,328,dimethyl peroxide,Organic peroxides,Other peroxides,Other peroxides,270.414,11.08210,NIST,COOC,C2H6O2,...,2,0,117.440572,1.052965,0.046742,-2.374345,-11.250341,5.997508,0.148167,0.500000
3,329,heptane-3-peroxol,Organic peroxides,Hydroperoxides,Aliphatic hydroperoxides,459.704,1.50806,NIST,CCCCC(CC)OO,C7H16O2,...,2,0,171.625428,1.106192,0.031590,-4.779775,-13.715054,53.241940,0.329739,0.325457
4,330,propane-1-peroxol,Organic peroxides,Hydroperoxides,Aliphatic hydroperoxides,377.335,1.19363,NIST,CCCOO,C3H8O2,...,2,0,72.206321,1.061564,0.040008,-3.716518,-15.942061,15.945147,0.436619,0.431461


# Import molecular geometries from cartesian xyz coordinates

In [19]:
psi4_mols = []
for geo in xyzgeom_list:
    psi4mol = psi4.geometry(geo)
    psi4_mols.append(psi4mol)
    
E_list = []
wfn_list = []
homo_list = []
lumo_list = []
DM_list = []
const = psi4.constants.dipmom_au2debye #atomic units to Debye conversion factor for dipoles

# Molecular structure optimization round 2 + calculate HOMO, LUMO, and DM

In [20]:
psi4.set_options({'reference': 'uhf',
                   'geom_maxiter':350,
                    'opt_coordinates': 'cartesian'})
k=326
for molec in psi4_mols: #iterate over list of molecular geometries
    
    E, wfn= psi4.optimize("B3LYP/6-31G*", molecule = molec,return_wfn=True) #returns energy and wavefunction of molecule
    
    E_list.append(E)
    wfn_list.append(wfn)
    
    HOMO = ( np.array(wfn.epsilon_a_subset("AO", "ALL")) )[wfn.nalpha()-1]
    LUMO = ( np.array(wfn.epsilon_a_subset("AO", "ALL")) )[wfn.nalpha()]
    dipole_xyz = wfn.variable("SCF DIPOLE")
    dipole_debye = np.linalg.norm(dipole_xyz) *const
  
    homo_list.append(HOMO)
    lumo_list.append(LUMO)
    DM_list.append(dipole_debye)
    
    k+=1
    strID = "fuel"+str(k)+ ".xyz"
    molec.save_xyz_file(strID,1) #write final optimized geometry to XYZ file
    print("Fuel "+str(k)+" done, "+ "k: "+str(k))

Optimizer: Optimization complete!
Fuel 327 done, k: 327
Optimizer: Optimization complete!
Fuel 328 done, k: 328
Optimizer: Optimization complete!
Fuel 329 done, k: 329
Optimizer: Optimization complete!
Fuel 330 done, k: 330
Optimizer: Optimization complete!
Fuel 331 done, k: 331


# Add HOMO, LUMO, DM lists to dataframe and save as CSV file

In [21]:
df2 = pd.DataFrame(list(zip(homo_list,lumo_list,DM_list)), columns=['HOMO','LUMO','DM'])
df2

,HOMO,LUMO,DM
0,-0.254585,0.086322,1.843983
1,-0.271455,0.100516,1.478705
2,-0.243760,0.032669,1.394303
3,-0.253153,0.028305,1.664803
4,-0.249390,0.027843,1.741228


In [22]:
df3 = pd.concat([df,df2], axis=1)
df3

,ID,Fuel,Class,Sub-Class,Group,BP2,Uncertainty,Source,SMILES,Molecular Formula,...,GATS2m,FPSA3,WNSA3,PNSA3,ATS3p,RPCG,RNCG,HOMO,LUMO,DM
0,326,"1,1-dimethoxyethane",Acetals,Aliphatic acetals,Alkyl acetals,336.695,1.85029,NIST,COC(C)OC,C4H10O2,...,1.128388,0.038517,-2.858090,-10.848476,21.707281,0.211493,0.490707,-0.254585,0.086322,1.843983
1,327,"2,4,6-trimethyl-1,3,5-trioxane",Acetals,Aliphatic acetals,Alkyl acetals,397.316,1.84010,NIST,CC1OC(C)OC(C)O1,C6H12O3,...,0.912466,0.030933,-3.920698,-12.372305,41.060026,0.158624,0.320290,-0.271455,0.100516,1.478705
2,328,dimethyl peroxide,Organic peroxides,Other peroxides,Other peroxides,270.414,11.08210,NIST,COOC,C2H6O2,...,1.052965,0.046742,-2.374345,-11.250341,5.997508,0.148167,0.500000,-0.243760,0.032669,1.394303
3,329,heptane-3-peroxol,Organic peroxides,Hydroperoxides,Aliphatic hydroperoxides,459.704,1.50806,NIST,CCCCC(CC)OO,C7H16O2,...,1.106192,0.031590,-4.779775,-13.715054,53.241940,0.329739,0.325457,-0.253153,0.028305,1.664803
4,330,propane-1-peroxol,Organic peroxides,Hydroperoxides,Aliphatic hydroperoxides,377.335,1.19363,NIST,CCCOO,C3H8O2,...,1.061564,0.040008,-3.716518,-15.942061,15.945147,0.436619,0.431461,-0.249390,0.027843,1.741228


In [23]:
df3.to_csv('ExtraDescriptors.csv', encoding='utf-8', index=False)